# Group analysis (Gaussian and DoG)

In [3]:
from linescanning import (
    prf,
    optimal,
    pycortex,
    plotting,
    fitting,
    glm,
    utils
)
from prfpy import model, rf, timecourse
from prfpy.stimulus import PRFStimulus2D
from prfpy.stimulus import *
from prfpy.fit import *
from prfpy.model import *
from prfpy import timecourse
import matplotlib.pyplot as plt
import os
import numpy as np
import cortex
opj = os.path.join
opd = os.path.dirname
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.weightstats import DescrStatsW, ttost_paired
from statsmodels.stats.descriptivestats import Description
from seaborn import regression
import statsmodels.api as sm   

## Functions 

### Profile and add. measures (fwhmax, fwatmin, half max, min profile)

In [4]:
## Profile and add. measures (fwhmax, fwatmin, half max, min profile)
class Add_measures():
    
    'To obtain profile, fwhmax, halfmax, fwatmin, min profile'

    def __init__(
        self, 
        data = None, 
        model = None,
        ecc = True,
        #params=None,  
        # roi = None,
        normalize_RFs = False
        ):
            
        self.data = data
        self.model = model
        #self.params = params
        #self.roi = roi
        self.normalize_RFs = normalize_RFs
        self.ecc = ecc
        
        
        self.fwhmax = []
        self.fwhmin = []
        self.halfmax = []
        self.minprofile = [] 
        self.profile_df = pd.DataFrame()
        
        if self.ecc:
            self.data = self.data[(self.data.ecc > 0.5) & (self.data.ecc < 4.51)] # not including those vooxels with higher ecc - not even for the profile
        
        for index in range(len(self.data)):    
            fwhmaxx, halfmaxx, fwhminn,  minprofilee, profile , self.x = self.fwhmax_fwatmin(model = self.model, 
                                                                                   params = self.data.iloc[index], 
                                                                                   normalize_RFs = self.normalize_RFs)  # fwhmax, half_max, [], [], list(profile.T), x
            
            self.fwhmax.append(float(fwhmaxx))
            self.halfmax.append(float(halfmaxx))
            self.profile_df[index] = profile
            
            if self.model == 'dog':
                self.minprofile.append(float(minprofilee))
                self.fwhmin.append(float(fwhminn))
            
        
        self.data['fwhmax']= self.fwhmax
        self.data['halfmax'] = self.halfmax

        if self.model == 'dog':
            self.data['fwmin'] = self.fwhmin
            self.data['min_profile'] = self.minprofile
            
        
        
    
    def fwhmax_fwatmin(self, model, params, normalize_RFs):
    
        model = model.lower()
        x = np.linspace(-20,20,1000).astype('float32')

        prf = params.prf_ampl * np.exp(-0.5*x[...,np.newaxis]**2 / params.prf_size**2) #params[...,3] * np.exp(-0.5*x[...,np.newaxis]**2 / params[...,2]**2)
        vol_prf =  2*np.pi*params.prf_size**2

        if 'dog' in model or 'norm' in model:
            srf = params.surr_ampl * np.exp(-0.5*x[...,np.newaxis]**2 / params.surr_size**2)
            vol_srf = 2*np.pi*params.surr_size**2

        if normalize_RFs==True:

            if model == 'gauss':
                profile =  prf / vol_prf

            elif model =='dog':
                profile = prf / vol_prf - \
                        srf / vol_srf
        else:
            if model == 'gauss':
                profile = prf
            elif model =='dog':
                profile = prf - srf

        half_max = np.max(profile, axis=0)/2
        
        fwhmax = np.abs(2*x[np.argmin(np.abs(profile-half_max), axis=0)])


        if 'dog' in model or 'norm' in model:

            min_profile = np.min(profile, axis=0)
            
            fwatmin = np.abs(2*x[np.argmin(np.abs(profile-min_profile), axis=0)])

            return fwhmax, fwatmin, half_max, min_profile, list(profile.T), x
        else:
            return fwhmax, half_max, [], [], list(profile.T), x
    
class Add_measures_2d():
    
    'To obtain profile, fwhmax, halfmax, fwatmin, min profile'

    def __init__(
        self, 
        data = None, 
        model = None,
        ecc = True,
        #params=None,  
        # roi = None,
        normalize_RFs = False
        ):
            
        self.data = data
        self.model = model
        #self.params = params
        #self.roi = roi
        self.normalize_RFs = normalize_RFs
        self.ecc = ecc
        
        self.design, self.prf_stim = self.load_dm()
        
        self.fwhmax = []
        self.fwhmin = []
        self.halfmax = []
        self.minprofile = [] 
        self.profilex_df = pd.DataFrame()
        self.profiley_df = pd.DataFrame()
        
        if self.ecc:
            self.data = self.data[(self.data.ecc > 0.5) & (self.data.ecc < 4.51)] # not including those vooxels with higher ecc - not even for the profile
        
        for index in range(len(self.data)):    
            fwhmaxx, halfmaxx, fwhminn,  minprofilee, profilex, profiley , self.x = self.fwhmax_fwatmin(model = self.model, 
                                                                                   params = self.data.iloc[index], 
                                                                                   normalize_RFs = self.normalize_RFs, prf_stim = self.prf_stim)  # fwhmax, half_max, [], [], list(profile.T), x
            
            self.fwhmax.append(float(fwhmaxx))
            self.halfmax.append(float(halfmaxx))
            self.profilex_df[index] = profilex
            self.profiley_df[index] = profiley
            
            if self.model == 'dog':
                self.minprofile.append(float(minprofilee))
                self.fwhmin.append(float(fwhminn))
            
        
        self.data['fwhmax']= self.fwhmax
        self.data['halfmax'] = self.halfmax

        if self.model == 'dog':
            self.data['fwmin'] = self.fwhmin
            self.data['min_profile'] = self.minprofile
            
        
        
    
    def fwhmax_fwatmin(self, model, params, normalize_RFs, prf_stim):
    
        model = model.lower()
        x = np.linspace(-20,20,1000).astype('float32')

        prf = params.prf_ampl * np.exp(-0.5*x[...,np.newaxis]**2 / params.prf_size**2) #params[...,3] * np.exp(-0.5*x[...,np.newaxis]**2 / params[...,2]**2)
        vol_prf =  2*np.pi*params.prf_size**2

        if 'dog' in model or 'norm' in model:
            srf = params.surr_ampl * np.exp(-0.5*x[...,np.newaxis]**2 / params.surr_size**2)
            vol_srf = 2*np.pi*params.surr_size**2

        if normalize_RFs==True:

            if model == 'gauss':
                profile =  prf / vol_prf

            elif model =='dog':
                profile = prf / vol_prf - \
                        srf / vol_srf
        else:
            if model == 'gauss':
                profile = prf
            elif model =='dog':
                profile = prf - srf

        half_max = np.max(profile, axis=0)/2
        
        fwhmax = np.abs(2*x[np.argmin(np.abs(profile-half_max), axis=0)])

        if 'dog' in model or 'norm' in model:

            min_profile = np.min(profile, axis=0)
            
            fwatmin = np.abs(2*x[np.argmin(np.abs(profile-min_profile), axis=0)])
            
            profilex = self.make_prf(model = model, stim = prf_stim, params = params)[0]
            profiley = self.make_prf(model = model, stim = prf_stim, params = params)[1]

            return fwhmax, fwatmin, half_max, min_profile, list(profilex.T), list(profiley.T), x
        else:
            profilex = self.make_prf(model = model, stim = prf_stim, params = params)[0]
            profiley = self.make_prf(model = model, stim = prf_stim, params = params)[1]
            return fwhmax, half_max, [], [], list(profilex.T), list(profiley.T), x
        
    
    def make_prf(self, model, stim, params, mu_x=0, mu_y=0, size=None, resize_pix=None, **kwargs):
        """make_prf

        Create an instantiation of a pRF using the parameters obtained during fitting.

        Parameters
        ----------
        prf_object: prfpy.stimulus.PRFStimulus2D
            representation the pRF in visual space
        mu_x: float
            x-component of pRF. Leave default if you want to create size/response functions
        mu_y: float
            y-component of pRF. Leave default if you want to create size/response functions
        size: float
            size of pRF, optional
        resize_pix: int
            resolution of pRF to resample to. For instance, if you've used a low-resolution design matrix, but you'd like a prettier image, you can set `resize` to something higher than the original (54 >> 270, for example). By default not used.

        Returns
        ----------
        numpy.ndarray
            meshgrid containing Gaussian characteristics of the pRF. Can be plotted with :func:`linescanning.plotting.LazyPRF`

        prf = np.rot90(rf.gauss2D_iso_cart(
            x=prf_object.x_coordinates[..., np.newaxis],
            y=prf_object.y_coordinates[..., np.newaxis],
            mu=(mu_x, mu_y),
            sigma=size,
            normalize_RFs=False).T, axes=(1, 2))
        """
        
        prf = params.prf_ampl[...,np.newaxis,np.newaxis]*np.rot90(
            rf.gauss2D_iso_cart(
                x=stim.x_coordinates[...,np.newaxis],
                y=stim.y_coordinates[...,np.newaxis],
                mu=(params.x, params.y),
                sigma=params.prf_size,
                normalize_RFs=False).T, axes=(1,2))
        
        if model == 'dog':
        
            prf -= params.surr_ampl[...,np.newaxis,np.newaxis]*np.rot90(
                    rf.gauss2D_iso_cart(
                        x=stim.x_coordinates[...,np.newaxis],
                        y=stim.y_coordinates[...,np.newaxis],
                        mu=(params.x, params.y),
                        sigma=params.surr_size,
                        normalize_RFs=False).T, axes=(1,2))
            
        #print(x, y)

        # spatially smooth for visualization
        if isinstance(resize_pix, int):
            prf_squeezed = np.squeeze(prf, axis=0)
            prf = utils.resample2d(prf_squeezed, resize_pix)
        
        # save a bunch of problems if returned array is 2D
        if prf.ndim > 2:
            prf = np.squeeze(prf,axis=0)
            
        return prf
    
    
    def load_dm(cut_volumes = True):

        #### load design matrix ###
        screen_size_cm =39.3
        screen_distance_cm=210
        grid_nr = 20
        TR= 1.5
        
        if cut_volumes:
            n_volumes = 5
        else:
            n_volumes = 0

        design = prf.read_par_file(opj(opd(opd(prf.__file__)), '/data1/projects/Meman1/projects/pilot/code', 'design_task-2R.mat'))

        prf_stim = PRFStimulus2D(screen_size_cm = screen_size_cm,
                                    screen_distance_cm = screen_distance_cm,
                                    design_matrix = design[:, :, n_volumes:], # remove first 5 volumes
                                    TR = TR,
                                    task_names ='2R')
        
        return design, prf_stim
    
def load_dm(cut_volumes = True):

    #### load design matrix ###
    screen_size_cm =39.3
    screen_distance_cm=210
    grid_nr = 20
    TR= 1.5
    
    if cut_volumes:
        n_volumes = 5
    else:
        n_volumes = 0

    design = prf.read_par_file(opj(opd(opd(prf.__file__)), '/data1/projects/Meman1/projects/pilot/code', 'design_task-2R.mat'))

    prf_stim = PRFStimulus2D(screen_size_cm = screen_size_cm,
                                screen_distance_cm = screen_distance_cm,
                                design_matrix = design[:, :, n_volumes:], # remove first 5 volumes
                                TR = TR,
                                task_names ='2R')
    
    return design, prf_stim

### Load parameters ROI

In [5]:
## Load parameters ROIs
def load_params_roi(sub, roi, model, cv):
    
    if roi != 'v1':
        roi = roi.upper()
    if cv:
        add = '_train-test'
    else:
        add = ''
    
    ### SESSION 2
    prf_fn_s2 = opj("/data1/projects/Meman1/projects/pilot/derivatives/prf",f'sub-{sub}', f"ses-2{add}", f"sub-{sub}_ses-2_task-2R_roi-{roi}_model-{model}_stage-iter_desc-prf_params.pkl") #f"sub-{sub}_ses-2_task-2R_roi-{roi}_model-{model}_stage-iter_desc-prf_params.pkl"
    
    pars_s2 = prf.Parameters(prf.read_par_file(prf_fn_s2), model=model).to_df() #gettig prf parameters 
    #data_s2 = prf.Parameters(prf.read_par_file(prf_fn_s2), model=model).to_df() #gettig prf parameters 
    data_s2 = pars_s2.loc[pars_s2.r2 != 0] 
    #data_s2 = pars_s2.loc[(pars_s2.ecc>0) & (pars_s2.ecc<=5)] # selecting voxels which are true for V1 (whole roi) and ecc higher than 0
    
    data_s2['roi'] = roi
    
    data_s2['session'] = 2
    
    if cv:
        cvrsq_s2 = pd.read_csv(f'/data1/projects/Meman1/projects/pilot/derivatives/prf/sub-{sub}/ses-2{add}/rsqs_{roi}_2_sub{sub}.csv')
        cvrsq_s2.index = data_s2.index
    else:
        cvrsq_s2 = []
    
    ### SESSION 3
    
    prf_fn_s3 = opj("/data1/projects/Meman1/projects/pilot/derivatives/prf",f'sub-{sub}', f"ses-3{add}", f"sub-{sub}_ses-3_task-2R_roi-{roi}_model-{model}_stage-iter_desc-prf_params.pkl") #f"sub-{sub}_ses-2_task-2R_roi-{roi}_model-{model}_stage-iter_desc-prf_params.pkl"
    
    pars_s3 = prf.Parameters(prf.read_par_file(prf_fn_s3), model=model).to_df() #gettig prf parameters 
    #data_s3 = prf.Parameters(prf.read_par_file(prf_fn_s3), model=model).to_df() #gettig prf parameters 
            
    data_s3 = pars_s3.loc[pars_s3.r2 != 0] 
    # data_s3 = pars_s3.loc[(pars_s3.ecc>0) & (pars_s3.ecc<=5)] # selecting voxels which are true for V1 (whole roi) and ecc higher than 0

    data_s3['roi'] = roi
    
    data_s3['session'] = 3
    
    if cv:
        cvrsq_s3 = pd.read_csv(f'/data1/projects/Meman1/projects/pilot/derivatives/prf/sub-{sub}/ses-3{add}/rsqs_{roi}_3_sub{sub}.csv')
        cvrsq_s3.index = data_s3.index
    else:
        cvrsq_s3 = []
    
    if sub in ['003', '004', '007', '012', '016']: # meman session 2
        
        data_s2['ses'] = 'memantine'
        
        data_s3['ses'] = 'placebo'
        
    else:  ## meman in session 3     
        
        data_s3['ses'] = 'memantine'
        
        data_s2['ses'] = 'placebo'

    data = pd.concat([data_s2, data_s3])
    
    data['subject'] = sub
    
    if cv:
        data['rsq_test'] = pd.concat([cvrsq_s2['tc_rsq_test'], cvrsq_s3['tc_rsq_test']])
    
    dataf = Add_measures(data = data, model = model, normalize_RFs = False, ecc = True)
    
    
    #dataf = fwhmax_fwhmin_allvoxels(data = data, model = 'dog', normalize_RFs = False) # calculate fwhmax, fwhmin
    
    #datae = dataf.data[dataf.data.ecc < 5.5]
    
    #datae = dataf[dataf.ecc<5.5]
    
    dataf.data.to_csv(f'/data1/projects/Meman1/projects/pilot/derivatives/prf/sub-{sub}/sub-{sub}_roi-{roi}_{model}_parameters.csv')
    
    # dataf.data contains the whole dataset
    # dataf.profile contains the profile for each voxel
    
    return dataf, cvrsq_s2, cvrsq_s3

#### Load all sbjects class

In [6]:
class AllSubjParams():
    
    '''
    Class to fit specific dog parameter across eccentricity bins for all subjects, using a weighted r2 mean or not (w = True /False) and plot it
        
        y: can be 'yfit'(fitted bins to intercept and slope, estimated using all data) or 'y' (actual data binned)
        
        w: use the weighted mean for r2 or just the mean of the bin . if W = false : median instead of mean is used and no weighted by R2
        
        testprint : show posthoc test parameters or not - it is a ttest using FDR correction
                    ## FDR-corrected post hocs with Cohen's D effect size self.posthoc = pingouin.pairwise_tests(
                    ## uses the parametric ttest() function. If False, use pingouin.wilcoxon() or pingouin.mwu() for paired or unpaired samples, respectively.
                        if 0.01 < p_val < 0.05:txt = "*"
                        elif 0.001 < p_val < 0.01:txt = "**"
                        elif p_val < 0.001:txt = "***"
                    
    It returns a barplot with error of all parameters across both sessions and the points values (bin values). It shows whether differences between sessions for specific parameter are significant or not.
                            
    '''
        
    def __init__(
        self, 
        data=None, 
        par=None, 
        #roi=None, 
        nbins=None, 
        y=None, 
        w=None, 
        figsize=(4,8),
        testprint=False, 
        axs=None,
        **kwargs):
    
    
        ## unify all subjects fitting parameter ###
        
        self.data = data
        self.par = par
        self.rois = data.roi.unique()
        self.nbins = nbins
        self.y = y
        self.w = w
        self.testprint = testprint
        self.figsize = figsize
        self.axs = axs
        self.subjects = data.subject.unique()

        self.regdf = pd.DataFrame()
        
        for sub in self.subjects:
            for roi in self.rois:
                #_, _, self.regr_df, self.x = self.fit_param_bins(
                
                #self.regr_df, self.x = self.fit_param_binss(
                _, _, self.regr_df, self.x = self.fit_param_bins(
                    sub, 
                    roi, 
                    self.par, 
                    self.data[self.data.subject == sub], 
                    self.nbins, 
                    self.w)  
                
                self.regdf = pd.concat([self.regdf, self.regr_df])
                
            self.df = self.regdf[self.regdf.parameter == self.par]
        
        
        if not "ses_short" in list(self.df.columns):
            ses_short = self.df["ses"].values.copy()
            ses_short = np.array([i[:3] for i in ses_short])

            self.df["ses_short"] = ses_short
        
 
        
    def fit_param_bins(self, sub, roi, par, data, nbins, w):
        
        '''
        Function to fit specific dog parameter across eccentricity bins, using a weighted r2 mean or not (w = True /False)
        
        if W = false : median instead of mean is used and no weighted by R2
        '''
        
        binpar ='ecc'#'std_prf_ecc'#'std_size'
        b = np.linspace(0.5, 4.5, nbins)
        x = np.asarray([(b[i]+b[i+1])/2 for i in range(len(b)-1)])
        
        ### data for each session
        s0 = data[(data.roi == roi) & (data.ses == 'placebo')] # data for placebo session(data.subject == sub) & 
        s1 = data[(data.roi == roi) &  (data.ses == 'memantine')] # data for memantine session(data.subject == sub)

    
        ### fitting it to a linear function - estimating intercept and slope ###
        p0 = np.polyfit(s0[binpar], s0[par], 1) # placebo - fitting it to a linear function - finding intercept and slope
        p1 = np.polyfit(s1[binpar], s1[par], 1) # memantine, full = True   - fitting it to a linear function - finding intercept and slope
        
        S0 = sm.add_constant(s0[binpar])
        S1 = sm.add_constant(s1[binpar])
        wp0 = sm.WLS(s0[par], S0, weights=s0['r2']).fit()
        wp1 = sm.WLS(s1[par], S1, weights=s1['r2']).fit()
        #print(wp0.fittedvalues)
        #print(wp0.params)
        
        wyfit0 = np.polyval(wp0.params, x)
        wyfit1 = np.polyval(wp1.params, x)

        ### calculating predicted values ###
        yfit0 = np.polyval(p0, x)
        yfit1 = np.polyval(p1, x)
        
        
        ### get mean values for each bin ###
        y0, std0 = self.return_binned(s0, 
                                        s0[par], 
                                        s0[binpar],
                                        w, 
                                        list(b)) # mean value for the bin - real data

        y1, std1 = self.return_binned(s1, 
                                      s1[par], 
                                      s1[binpar], 
                                      w, 
                                      list(b)) # mean value for each bin - real data
        
        yfit0 = np.polyval(p0, x)
        yfit1 = np.polyval(p1, x)
        
        ### create df with fitted values ###
        
        # df = pd.DataFrame({"fit": np.concatenate(tmp_list), "ix": np.concatenate(tmp_ix), "ses": np.concatenate(tmp_name), "subject": np.concatenate(tmp_subj)})

        regr_df = pd.DataFrame()
        std1 = np.array(std1)
        std0 = np.array(std0)
        regr_df['yfit'] = np.hstack([yfit0, yfit1])
        regr_df['wyfit'] = np.hstack([wyfit0, wyfit1])
        regr_df['y'] = np.hstack([y0, y1])
        regr_df['std'] = np.hstack([std0, std1]) #std of the bin mea    
        #regr_df['ci_l'] = np.hstack([ci_l0, ci_l1]) #std of the bin mean
        regr_df['ses'] = np.hstack([np.repeat('placebo', nbins-1), np.repeat('memantine', nbins-1)])
        regr_df['subject'] = np.repeat(sub, nbins*2-2)
        regr_df['intercept'] = np.hstack([np.repeat(p0[0], nbins-1), np.repeat(p1[0], nbins-1)])    
        regr_df['slope'] = np.hstack([np.repeat(p0[1], nbins-1), np.repeat(p1[1], nbins-1)])
        regr_df['wintercept'] = np.hstack([np.repeat(wp0.params[0], nbins-1), np.repeat(wp1.params[0], nbins-1)])    
        regr_df['wslope'] = np.hstack([np.repeat(wp0.params[1], nbins-1), np.repeat(wp1.params[1], nbins-1)])
        regr_df['diff_fit'] = np.hstack([yfit1-yfit0, yfit1-yfit0]) # memantine - placebo (putting it twice to adjust in df
        regr_df['diff_y'] = np.hstack([y1-y0, y1-y0]) # memantine - placebo (putting it twice to adjust in df
        regr_df['diff_std'] = np.hstack([np.abs(std1-std0), np.abs(std1-std0)]) # memantine - placebo (putting it twice to adjust in df
        regr_df['roi'] = roi
        regr_df['parameter'] = par

        std_resid0 = np.std(y0-yfit0)
        std_resid1 = np.std(y1-yfit1)

        return p0, p1, regr_df, x
    
    
    def fit_param_binss(self, roi, par, data, nbins, w):
        
        '''
        Function to fit specific dog parameter across eccentricity bins, using a weighted r2 mean or not (w = True /False)
        
        We are obtaining the CI of each fit using seaborn regression package
        
        No weighted mean is used here (we only need the fitted differences)
        
        The grid is performed tiny differently (it has 20 values instead of 18 as above)
        '''
        
        binpar ='ecc'#'std_prf_ecc'#'std_size'
        
        b = np.linspace(0.5, 4.5, nbins)
        
        ### data for each session
        s0 = data[(data.roi == roi) & (data.ses == 'placebo')] # data for placebo session(data.subject == sub) & 
        s1 = data[(data.roi == roi)  & (data.ses == 'memantine')] # data for memantine session& (data.subject == sub)

    
        ### fitting it to a linear function - estimating intercept and slope ###
        
        #p0 = np.polyfit(s0[binpar], s0[par], 1) # placebo - fitting it to a linear function - finding intercept and slope

        #p1 = np.polyfit(s1[binpar], s1[par], 1) # memantine, full = True   - fitting it to a linear function - finding intercept and slope

        reg_pla = regression._RegressionPlotter(data = s0, x = 'ecc',  y=par,  x_bins = nbins
                                                , x_ci = None)
        
        reg_mem = regression._RegressionPlotter(data = s1, x = 'ecc',  y=par,  x_bins = nbins
                                                , x_ci = None)

        grid, yhat_pla, err_bands_pla = reg_pla.fit_regression(grid = b) #  return grid, yhat, err_bands
        grid, yhat_mem, err_bands_mem = reg_mem.fit_regression(grid = b) #  return grid, yhat, err_bands

        regr_df = pd.DataFrame()
        regr_df['yfit'] = np.hstack([yhat_pla, yhat_mem])
        regr_df['ci_l'] = np.hstack([err_bands_pla[0], err_bands_mem[0]]) #std of the bin mean
        regr_df['ci_u'] = np.hstack([err_bands_pla[1], err_bands_mem[1]]) #std of the bin mean
        regr_df['ses'] = np.hstack([np.repeat('placebo', nbins), np.repeat('memantine', nbins)])
        #regr_df['subject'] = np.repeat(sub, nbins*2)
        regr_df['diff_fit'] = np.hstack([yhat_mem-yhat_pla, yhat_mem-yhat_pla]) # memantine - placebo (putting it twice to adjust in df)
        regr_df['diff_cil'] = np.hstack([err_bands_mem[0]-err_bands_pla[0], err_bands_mem[0]-err_bands_pla[0]])  # memantine - placebo (putting it twice to adjust in df)
        regr_df['diff_ciu'] = np.hstack([err_bands_mem[1]-err_bands_pla[1], err_bands_mem[1]-err_bands_pla[1]])  # memantine - placebo (putting it twice to adjust in df)
        regr_df['roi'] = roi
        regr_df['parameter'] = par
    
        return regr_df, grid
    
    
    def return_binned(self, data, data_par, binon, w, bins=None, start=None, stop=None, binsize=None, nbins=None):
        
        """
        Bins data according to provided bins. Can provide either a list of bins,
        or start, stop and bin sizes or nbins.

        Equivalent to: scipy.stats.binned_statistic()

        data (1D array) : values to be averaged within a bin
        binon (1D array) : independent values to bin-on (separate from values being averaged)
        bins (list) : list of bins
        start (float) :
        stop (float) :
        binsize (float) :
        nbins (int) :

        Returns
        binned_data (arr)
        
        if W = false : median instead of mean is used and no weighted by R2
        using the wighted mean by r2, the fitting is much closer to it
        
        median is much closer to the fitting as well - avoid outliers having too much influence 
        
        fitting is performed to all data - no bins
        """

        #data=np.asarray(data_par)
        binon=np.asarray(binon)

        assert data_par.ndim,'data must be 1D'

        if not bins and binsize:
            assert start and stop, 'provid list of bins or start and stop values'
            bins=np.arange(start,stop,binsize)
        elif not bins and nbins:
            assert start and stop, 'provid list of bins or start and stop values'
            bins=np.linspace(start,stop,nbins)
        else:
            assert bins, 'provide bins'

        binned = [data_par[(binon >= bins[i]) & (binon<bins[i+1])].median() for i in range(len(bins)-1)] # data=data[par]
        #print(binned)
        std = [data_par[(binon >= bins[i]) & (binon<bins[i+1])].std() for i in range(len(bins)-1)] # not very useful if using the median but good to check outliers?
        
        
        
        if w: # calculate the mean of each bin (made by eccentricity size) by r2 weighted - those voxels with higher explainable variance will have more weight in the mean
            binned = [DescrStatsW(data = data_par[(binon >= bins[i]) & (binon<bins[i+1])], weights = data[(binon >= bins[i]) & (binon<bins[i+1])].r2).mean for i in range(len(bins)-1)]
            std = [DescrStatsW(data = data_par[(binon >= bins[i]) & (binon<bins[i+1])], weights = data[(binon >= bins[i]) & (binon<bins[i+1])].r2).std_mean for i in range(len(bins)-1)]
        
        return np.asarray(binned), std

#### Weighted mean profile

In [7]:
def wmean_profile(param_sub, roi):
    
    '''param_sub: is the object created that returns the data with all parameters and the profile
    
    It computes weighted mean by r2 of the profiles of the subject (point by point) and not the parameter which can be affected by outliers/etc
    '''

    profile_sub = param_sub.profile_df.T[0].apply(pd.Series)  # 10000 values - 
    #convert each point calculated into a column in df

    profile_sub.index = param_sub.data.index # get same indexes to locate
    
    profile_sub['r2'] = param_sub.data.r2

    pla_profile = []
    mem_profile = []
    std_mem = []
    std_pla = []


    for point in range(1000):
        print(profile_sub[param_sub.data.ses == 'placebo'][point])
        print(profile_sub.r2) 
                                                
        std_pla.append(DescrStatsW(data = profile_sub[param_sub.data.ses == 'placebo'][point], 
                            weights = profile_sub[param_sub.data.ses == 'placebo'].r2).std_mean )# standard deviation of the weighted mean
                            
        std_mem.append(DescrStatsW(data = profile_sub[param_sub.data.ses == 'memantine'][point], 
                            weights = profile_sub[param_sub.data.ses == 'memantine'].r2).std_mean) # standard deviation of the weighted mean
        
        pla_profile.append( DescrStatsW(data = profile_sub[param_sub.data.ses == 'placebo'][point], 
                            weights = profile_sub[param_sub.data.ses == 'placebo'].r2).mean )
        
        mem_profile.append( DescrStatsW(data = profile_sub[param_sub.data.ses == 'memantine'][point], 
                            weights = profile_sub[param_sub.data.ses == 'memantine'].r2).mean )
    
    profile = pd.DataFrame({'ses': np.array(param_sub.data.ses.unique()), 
                            'subject': np.hstack([param_sub.data.subject.unique(), param_sub.data.subject.unique()]),
                            'roi': [roi, roi]})
    
    profile = pd.concat([profile, pd.DataFrame([pla_profile, mem_profile])], axis = 1)
    
    return profile, std_pla, std_mem

## Load all parameters

In [ ]:
rois = ['v1', 'v2', 'v3']
subjects = [ '001','005','007', '008', '010', '012'] 
model = 'dog'
cv = False

parameters_subs = pd.DataFrame()
profiles_subs = pd.DataFrame()

for sub in subjects:
    print(sub)
    
    for roi in rois:
        print(roi)
        
        param_sub, _, _ = load_params_roi(sub, roi, model, cv)
    
        parameters_subs = pd.concat([parameters_subs, param_sub.data], axis = 0) #parameters
        
        profile, std_pla, std_mem = wmean_profile(param_sub, roi) # weighted mean of profiles (not parameters)    
        profiles_subs = pd.concat([profiles_subs, profile], axis = 0)

        # plotting.LazyPlot(
        #     ts = [profile.loc[1][3:],
        #           profile.loc[0][3:]],
        #     xx = param_sub.x, # I could reduce it
        #     color = ['indianred', 'black'],
        #     labels = ['memantine', 'placebo'],
        #     title = f'Mean profile sub-{sub} ({roi.upper()})',
        #     x_label = 'x (deg of visual field)',
        #     y_label = f'amplitude - {model} model',
        #     figsize = [10,10],
        #     #markers = [None, '.'],
        #     line_width  = [3,2],
        #     font_size = 30,
        #     #save_as = f"/data1/projects/Meman1/projects/pilot/code/results_images/medianprofile_sub-{sub}_roi-{roi}.jpg"
        #     save_as = f"/data1/projects/Meman1/projects/pilot/code/results_images/wmeanprofile_sub-{sub}_roi-{roi}_{model}.jpg"
            
        # )
        
print(np.unique(parameters_subs.subject))

parameters_subs.to_csv(f'all_subjects_parameters_{model}.csv')

In [10]:
dog_parameters = pd.read_csv('/data1/projects/Meman1/projects/pilot/code/def_code/all_subjects_parameters_dog.csv')

gauss_parameters = pd.read_csv('/data1/projects/Meman1/projects/pilot/code/def_code/all_subjects_parameters_dog.csv')

## Profile

In [ ]:
%matplotlib inline
fig,axs = plt.subplots(nrows=3, figsize=(8,16), constrained_layout=True, sharey = True)

# ax1 = fig.add_subplot(gs[0])
# ax2 = fig.add_subplot(gs[1])
# ax3 = fig.add_subplot(gs[2])

for i,roi in enumerate(profiles_subs.roi.unique()):
    profile = profiles_subs[profiles_subs.roi == roi]
    
    mean_mem = profile[profile.ses == 'memantine']
    mean_pla = profile[profile.ses == 'placebo']  
    
    # df_ks = pd.DataFrame()
    # df_ks['Income'] = np.sort(df['Income'].unique())
    # df_ks['F_control'] = df_ks['Income'].apply(lambda x: np.mean(income_c<=x))
    # df_ks['F_treatment'] = df_ks['Income'].apply(lambda x: np.mean(income_t<=x))
    # df_ks.head() 
    
    plotting.LazyPlot(
                ts = [mean_mem.mean()[1:],
                    mean_pla.mean()[1:]],
                xx = param_sub.x, 
                color = ['indianred', 'black'],
                labels = ['memantine', 'placebo'],
                title = f'\n {roi.upper()}',
                #x_label = 'x ',
                y_label = f'amplitude',
                figsize = [4,15],
                axs = axs[i],
                add_hline = 0,
                #markers = ['.', '.'],
                line_width  = [2,1],
                font_size = 20,
                label_size = 17,
                x_lim = [-15,15],
                #y_lim = [-0.007,0.02],
                #save_as = f"/data1/projects/Meman1/projects/pilot/code/results_images/medianprofile_sub-{sub}_roi-{roi}.jpg"
                #save_as = f"/data1/projects/Meman1/projects/pilot/code/results_images/wmeanprofile_allsub_roi-{roi}_{model}.jpg"
                
            )
fig.suptitle('Weighted mean profile - DoG model',y = 1.05, fontsize = 20)
#fig.supylabel(f'amplitude' , fontsize = 13)
fig.supxlabel('x (deg. visual field)', fontsize = 20, y = -.05, x = .55) #
#plt.tight_layout()

### Distribution parameters

In [ ]:
#### DISTRIBUTION OF MAIN PARAMETERS ACROSS V1, SESSION, SUBJECT - eccentricity max set below 5.5####

parameters = ['fwhmax', 'fwmin', 'halfmax', 'ecc','r2' ]#'prf_size', 'prf_ampl','surr_size', 'surr_ampl']#,]#'min_profile',

g = plt.figure(figsize = [45,15])
axes = g.subplots(1,len(parameters))

for i, par in enumerate(parameters):
   
    sns.set(font_scale = 1.5)
    
    sns.set_style("whitegrid")

    sns.boxplot(y= par, x= "subject", hue='ses', data=parameters_subs,  ax = axes[i], palette =['#cccc', 'indianred']).set_title(par)

### Specific distribution for each subject

In [ ]:
#### SPECIFIC PARAMETER DISTRIBUTION FOR EACH SUBJECT ####

parameters = parameters_subs.columns.drop(['roi', 'session','subject', 'polar', 'ecc', 'r2', 'bold_bsl', 'x', 'y', 'ses'])

param = 'min_profile'

g = plt.figure(figsize = [25,10])
axes = g.subplots(1,5)

for subject, ax in zip(np.unique(parameters_subs.subject), axes.flatten()):
    
    sns.violinplot(y=param, x= "ses",  data=parameters_subs[parameters_subs.subject ==subject], ax=ax).set_title(f'{subject} - {param}')
    #axes.set_ylim([0,0.02])
    sns.despine(ax=ax)

plt.tight_layout()

### Dataset tatistics

In [ ]:
data_stats = parameters_subs.groupby(['subject', 'ses','roi']).describe().T

data_stats.to_csv(f'df_statistics_transpose_{model}.csv')

data_stats

## Effects of memantine

### FWHM

### Each subject fitted

In [ ]:
model = 'dog'

parameters_subs = pd.read_csv(f'all_subjects_parameters_{model}.csv')

data = parameters_subs
nbins =15
y = 'yfit'
w = True
par = 'fwmin'

regdf = AllSubjParams(
        data=data, 
        par=par, 
        #roi=roi, 
        nbins=nbins, 
        y=y, 
        w=w, 
        testprint=False,
        fancy=True,
        error="sem",
        sns_offset=4,
        #axs=axs[ix],
        fancy_denom=6,
        label_size=16,
        font_size=20) # points of plot are the bins

In [ ]:
#rois = regdf.df.roi.unique()
pla = regdf.df[regdf.df.ses=='placebo']
mem = regdf.df[regdf.df.ses=='memantine']
#fig,axs = plt.subplots(ncols=1, figsize=(19,8), constrained_layout=True, sharey = True)
parameter = 'fwmin'

sns.set_context('paper',font_scale=2.5)

a = sns.lmplot(data =  parameters_subs[(parameters_subs.ecc>=0.5) & (parameters_subs.ecc<=4.5)], x='ecc', 
                   y = parameter, row = 'subject', col= 'roi', hue = 'ses',  x_ci = None, palette =['grey','indianred' ], scatter = False, height= 8)

# axs.errorbar(regdf.x, mem[mem.roi== roi].y, yerr=mem[mem.roi== rois[0]]['std'], fmt="o", c='indianred')
#     axs[i].errorbar(regdf.x, pla[pla.roi== roi].y, yerr=pla[pla.roi== roi]['std'], fmt="o", c='#ccc')
#plt.suptitle('FWHM - DoG model \n \n \n', fontsize = 20)

for ix, sub in enumerate(parameters_subs.subject.unique()):
    for i,roi in enumerate(list(parameters_subs.roi.unique())):
        a.axes[ix][i].errorbar(regdf.x, pla[(pla.roi== roi) & (pla.subject== sub)].y, yerr=pla[(pla.roi== roi) & (pla.subject== sub)]['std'], fmt="o", c='grey', markersize = 10, capsize = 6)
        a.axes[ix][i].errorbar(regdf.x, mem[(mem.roi== roi) & (mem.subject== sub)].y, yerr=mem[(mem.roi== roi) & (mem.subject== sub)]['std'], fmt="o", c='indianred', markersize = 10, capsize = 6)

### Across all subjects

In [ ]:
model = 'gauss'

parameters_subs = pd.read_csv(f'all_subjects_parameters_{model}.csv')

data = parameters_subs
nbins =15
y = 'yfit'
w = True
par = 'fwhmax'

regdf = AllSubjParams(
        data=data, 
        par=par, 
        #roi=roi, 
        nbins=nbins, 
        y=y, 
        w=w, 
        testprint=False,
        fancy=True,
        error="sem",
        sns_offset=4,
        #axs=axs[ix],
        fancy_denom=6,
        label_size=16,
        font_size=20) # points of plot are the bins

In [ ]:
#rois = regdf.df.roi.unique()
pla = regdf.df[regdf.df.ses=='placebo']
mem = regdf.df[regdf.df.ses=='memantine']
#fig,axs = plt.subplots(ncols=1, figsize=(19,8), constrained_layout=True, sharey = True)
parameter = 'fwhmax'

sns.set_context('paper',font_scale=2.5)

a = sns.lmplot(data =  parameters_subs[(parameters_subs.ecc>=0.5) & (parameters_subs.ecc<=4.5)], x='ecc', 
                   y = parameter, col= 'roi', hue = 'ses',  x_ci = None, palette =['grey','indianred' ], scatter = False, height= 8)

# axs.errorbar(regdf.x, mem[mem.roi== roi].y, yerr=mem[mem.roi== rois[0]]['std'], fmt="o", c='indianred')
#     axs[i].errorbar(regdf.x, pla[pla.roi== roi].y, yerr=pla[pla.roi== roi]['std'], fmt="o", c='#ccc')
#plt.suptitle('FWHM - DoG model \n \n \n', fontsize = 20)


for i,roi in enumerate(list(parameters_subs.roi.unique())):
    a.axes[0][i].errorbar(regdf.x, [pla[pla.roi == roi]['y'].loc[i].mean() for i in range(14)], yerr=[pla[pla.roi == roi]['std'].loc[i].mean() for i in range(14)], fmt="o", c='grey', markersize = 10, capsize = 6)
    a.axes[0][i].errorbar(regdf.x, [mem[mem.roi == roi]['y'].loc[i].mean() for i in range(14,28)], yerr=[mem[mem.roi == roi]['std'].loc[i].mean() for i in range(14,28)], fmt="o", c='indianred', markersize = 10, capsize = 6)

#### Difference of fits

In [ ]:
rois = regdf.df.roi.unique()
pla = regdf.df[regdf.df.ses=='placebo']
mem = regdf.df[regdf.df.ses=='memantine']

plotting.LazyPlot(
   ts = [[pla[pla.roi == rois[0]]['diff_fit'].loc[i].mean() for i in range(14)] ,
         [pla[pla.roi == rois[1]]['diff_fit'].loc[i].mean() for i in range(14)] ,
         [pla[pla.roi == rois[2]]['diff_fit'].loc[i].mean() for i in range(14)]
         ],
   xx = regdf.x,
   x_label='eccentricity',
    y_label=f'diff. FWMIN',
    title = f'FWMIN (memantine - placebo)',
    labels = [ 'V1', 
              'V2', 
              'V3',
                ],
    cmap = 'inferno',
    #color = ['red','orange', 'yellow'],
    # line_width = [0.5, 2, 0.5, 
    #               0.5, 2, 0.5
    #               ,0.5, 2, 0.5,
    #               0.5, 2, 0.5, 
    #                0.5, 2, 0.5,
    #               0.5, 2, 0.5],
#     markers = [, 
#                '.', 
#                '.'],
    #alpha = [0.2, 1, 0.2],
    line_width = 5,
    figsize = [10,6],
    add_hline = 0,
    label_size = 20,
    save_as = f"/data1/projects/Meman1/projects/pilot/code/results_images/difffwhmax_allsub_{model}.jpg"
)
plt.tight_layout